The subscriber continuously receives the data. For each latest record r received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.

In [1]:
import paho.mqtt.client as mqtt
import pandas as pd
from io import StringIO
from sklearn.neighbors import KNeighborsClassifier

dataFrame= pd.DataFrame(columns=['Glucose','BloodPressure','Insulin','Outcome'])
knn5 = KNeighborsClassifier(n_neighbors=3)
lst_pred = []


def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe("myroom/diabetes")


def on_message(client, userdata, msg):
    global dataFrame, lst_pred, knn5
    new_record = pd.read_json(StringIO(msg.payload.decode()), typ='series')

    if len(dataFrame) > 5:
        knn5.fit(dataFrame[['Glucose','BloodPressure','Insulin']].tail(5), dataFrame['Outcome'].tail(5).astype(int))
        lst_pred = knn5.predict(new_record[['Glucose', 'BloodPressure', 'Insulin']].to_frame().T)

    print("Received myroom/diabetes: ", msg.payload)
    
    print('Prediction {}\r\n'.format(lst_pred))
    print('Actual:{}\r\n'.format(new_record['Outcome']))
    dataFrame.loc[len(dataFrame), dataFrame.columns] = new_record


        
    
# We create a client as the data subscriber and specify its actions for particular events
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

# Now, we connect to the data broker.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# As a simple example, we just keep the data listening/receiving on and on...
mqttc.loop_forever()

Connected with result code 0
Received myroom/diabetes:  b'{"Glucose": 0.9899497487437187, "BloodPressure": 0.5737704918032788, "Insulin": 0.6418439716312057, "Outcome": 1.0}'
Prediction []

Actual:1.0

Received myroom/diabetes:  b'{"Glucose": 0.628140703517588, "BloodPressure": 0.7868852459016393, "Insulin": 0.0, "Outcome": 1.0}'
Prediction []

Actual:1.0

Received myroom/diabetes:  b'{"Glucose": 0.5527638190954774, "BloodPressure": 0.7540983606557378, "Insulin": 0.0, "Outcome": 0.0}'
Prediction []

Actual:0.0

Received myroom/diabetes:  b'{"Glucose": 0.8442211055276382, "BloodPressure": 0.6065573770491803, "Insulin": 0.0, "Outcome": 1.0}'
Prediction []

Actual:1.0

Received myroom/diabetes:  b'{"Glucose": 0.6984924623115578, "BloodPressure": 0.6557377049180328, "Insulin": 0.0, "Outcome": 0.0}'
Prediction []

Actual:0.0

Received myroom/diabetes:  b'{"Glucose": 0.949748743718593, "BloodPressure": 0.49180327868852464, "Insulin": 1.0, "Outcome": 1.0}'
Prediction []

Actual:1.0

Received 

KeyboardInterrupt: 